Let's do a quick example comparing `jitr` to the standard Runge-Kutta ODE solver implemented in `scipy.integrate.solve_ivp`:

In [1]:
 import numpy as np
 from scipy.integrate import solve_ivp
 from numba import njit
 import jitr

In [2]:
 @njit
 def interaction(r, *args):
     (V0, W0, R0, a0) = args
     return jitr.woods_saxon_potential(r, V0, W0, R0, a0)

In [11]:
sys = jitr.ProjectileTargetSystem(
    np.array([939.0]),
    np.array([5 * (2 * np.pi)]),
    l=np.array([0]),
    Ztarget=40,
    Zproj=0,
    nchannels=1,
)

Ecom = 14.1

# Lagrange-Mesh solver
solver = jitr.LagrangeRMatrixSolver(100, 1, sys, ecom=Ecom)

# use same interaction for all channels
interaction_matrix = jitr.InteractionMatrix(1)
interaction_matrix.set_local_interaction(interaction)

# Woods-Saxon potential parameters
V0 = 60  # real potential strength
W0 = 20  # imag potential strength
R0 = 4  # Woods-Saxon potential radius
a0 = 0.5  # Woods-Saxon potential diffuseness

params = (V0, W0, R0, a0)

interaction_matrix.local_args[0,0] = params

In [12]:
ch = np.array(sys.build_channels(Ecom))
solver.set_energy(Ecom)

In [13]:
R, S, uext_prime_boundary = solver.solve(
    interaction_matrix, ch, ecom=Ecom
)

In [14]:
print(S[0][0])

(-0.056995279766827306-0.029788520925563822j)


Great, now let's use `scipy` and see if we get the same $\mathcal{S}$-matrix:

In [15]:
# Runge-Kutta
from jitr.utils import schrodinger_eqn_ivp_order1
sol_rk = solve_ivp(
    lambda s, y,: schrodinger_eqn_ivp_order1(
        s, y, ch[0], interaction_matrix.local_matrix[0, 0], params
    ),
    ch[0].domain,
    ch[0].initial_conditions(),
    dense_output=True,
    atol=1.0e-12,
    rtol=1.0e-12,
).sol

In [16]:
a = ch[0].domain[1]
R_rk = sol_rk(a)[0] / (a * sol_rk(a)[1])
S_rk = jitr.smatrix(R_rk, a, ch[0].l, ch[0].eta)
print(S_rk)

(-0.05699527983357116-0.029788521064808164j)


Great, this should be the same. Now let's compare the runtime of the two solver options:

In [17]:
%%timeit
R, S, uext_prime_boundary = solver.solve(
    interaction_matrix, ch, ecom=Ecom
)

3.34 ms ± 282 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
%%timeit
sol_rk = solve_ivp(
    lambda s, y,: schrodinger_eqn_ivp_order1(
        s, y, ch[0], interaction_matrix.local_matrix[0, 0], params
    ),
    (ch[0].domain[0], ch[0].domain[1]*1.2),
    ch[0].initial_conditions(),
    dense_output=True,
    atol=1.0e-12,
    rtol=1.0e-12,
).sol
a = ch[0].domain[1]
R_rk = sol_rk(a)[0] / (a * sol_rk(a)[1])
S_rk = jitr.smatrix(R_rk, a, ch[0].l, ch[0].eta)

487 ms ± 16.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


On my machine `jitr` is faster by around 200 times!